In [8]:
import brian2
from brian2.units import *
import numpy as np
import matplotlib.pyplot as plt
from math import dist

In [9]:
brian2.prefs.codegen.target = "numpy"

In [3]:
def D(i, j):
    i_pos = ((int)(i / 45), (i / 15) % 3, i % 15)
    j_pos = ((int)(j / 45), (j / 15) % 3, j % 15)
    return (dist(i_pos, j_pos))

In [28]:
brian2.start_scope()

tau_m = 30 * ms
R = 1 * Mohm
I_b = 13.5 * namp
G_1 = brian2.NeuronGroup(27, 'dv/dt = ( - v + R * I_b) / tau_m : volt',
                         threshold='v > 15*mV',
                         reset='v = 13.50*mV',
                         refractory='2 * ms',
                         method='euler')
G_2 = brian2.NeuronGroup(108, 'dv/dt = ( - v + 1 * Mohm * 13.56 * namp) / tau_m : volt',
                         threshold='v > 15*mV',
                         reset='v = 13.50*mV',
                         refractory='3 * ms',
                         method='euler')

lmbd = 1
synapse_model = '''u : 1
                   U : 1
                   d : second
                   f : second
                   r : 1
                   a : amp
                   lastupdate : second'''
synapse_on_pre = '''deltat = t - lastupdate
                    u_exp = u * exp(- deltat/f)
                    r_exp = exp(-deltat/d)
                    u = u_exp + U * (1 - u_exp)
                    r = r * (1 - u) * r_exp + 1 - r_exp
                    v_post += a * r * u * ohm
                    lastupdate = t'''
#D = lambda i, j : dist(((i // 45), (i // 15) % 3, i % 15), ((j // 45), (j // 15) % 3, j % 15))
cond_exp = f' * exp(-(1.0/{lmbd}) ** 2)'

S_exc_exc = brian2.Synapses(G_2, G_2,
                            model=synapse_model,
                            on_pre=synapse_on_pre,
                            delay=1.5*ms)
S_exc_exc.connect(condition=True,
                  p = '0.3' + cond_exp)

S_exc_inh = brian2.Synapses(G_2, G_1,
                            model=synapse_model,
                            on_pre=synapse_on_pre,
                            delay=0.8*ms)
S_exc_inh.connect(condition=True,
                  p = '0.2' + cond_exp)

S_inh_exc = brian2.Synapses(G_1, G_2,
                            model=synapse_model,
                            on_pre=synapse_on_pre,
                            delay=0.8*ms)
S_inh_exc.connect(condition=True,
                  p = '0.4' + cond_exp)

S_inh_inh = brian2.Synapses(G_1, G_1,
                            model=synapse_model,
                            on_pre=synapse_on_pre,
                            delay=0.8*ms)
S_inh_inh.connect(condition=True,
                  p = '0.1' + cond_exp)

"\nuse S.a = 'rand()', S.u = '...' for initial values\n"

In [34]:
M = brian2.SpikeMonitor(G_2, variables=['v'])
brian2.run(1000 * ms)
M.v

<spikemonitor_2.v: array([], dtype=float64) * volt>